In [1]:
#Import necessary modules
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Read the data
data = pd.read_excel('Assets/loadPrediction.xlsx')
data.head()

,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [3]:
#Divide the load data into columns L(T-3), L(T-2), L(T-1) and L(T)
x1 = []
x2 = []
x3 = []
y = []
data = data['Load (kW)']
for i in range(3, data.shape[0]):
    x1.append(data.iloc[i - 3])
    x2.append(data.iloc[i - 2])
    x3.append(data.iloc[i - 1])
    y.append(data.iloc[i])

In [4]:
#Create dataset with columns L(T-3), L(T-2), L(T-1), L(T)
data = pd.DataFrame({'L(T-3)' : x1, 'L(T-2)' : x2, 'L(T-1)' : x3, 'L(T)' : y})
data.head()

,L(T-3),L(T-2),L(T-1),L(T)
0,5551.82208,4983.17184,4888.39680,5072.95872
1,4983.17184,4888.39680,5072.95872,5196.25980
2,4888.39680,5072.95872,5196.25980,5641.29720
3,5072.95872,5196.25980,5641.29720,7153.17732
4,5196.25980,5641.29720,7153.17732,7849.80504


In [5]:
#Function to normalize a column
def normalize(column):
    c_max, c_min = np.max(column), np.min(column)
    for i in range(len(column)):
        column.iloc[i] = (column.iloc[i] - c_min) / (c_max - c_min)
    return c_max, c_min

#Normalize all columns of dataset and store max and min values for denormalizing later
x1max, x1min = normalize(data['L(T-3)'])
x2max, x2min = normalize(data['L(T-2)'])
x3max, x3min = normalize(data['L(T-1)'])
ymax, ymin = normalize(data['L(T)'])
data.head()

,L(T-3),L(T-2),L(T-1),L(T)
0,0.397877,0.293800,0.276454,0.310234
1,0.293800,0.276454,0.310234,0.332801
2,0.276454,0.310234,0.332801,0.414254
3,0.310234,0.332801,0.414254,0.690965
4,0.332801,0.414254,0.690965,0.818465


In [6]:
#Check for null values
data.isnull().sum()

L(T-3)    0
L(T-2)    0
L(T-1)    0
L(T)      0
dtype: int64

In [7]:
#Split data for testing and training
X = data[['L(T-3)', 'L(T-2)', 'L(T-1)']]
Y = data['L(T)']
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.1, random_state=101)

In [8]:
#Initialize Learning Rate, Epochs and Model Parameters
m = [np.random.uniform(-10, 10) for i in range(xtrain.shape[1])]
c = np.random.uniform(-10, 10)
eeta = 0.1
epochs = 500

In [9]:
##Train the model
for _ in range(epochs):
    for i in range(xtrain.shape[0]):
        y, ypred = ytrain.iloc[i], sum([xtrain.iat[i,z] * m[z] for z in range(xtrain.shape[1])]) + c
        gradm = [-1 * xtrain.iat[i,j] * (y - ypred) for j in range(xtrain.shape[1])]
        gradc = -1 * (y - ypred)
        c = c - eeta * gradc
        for j in range(len(m)):
            m[j] = m[j] - eeta * gradm[j]
            
#Display Model Parameters after training
print(m, c)

[-0.23128366799983094, -0.007745238430485762, 0.9080113730757413] 0.14143639495293966


In [10]:
#Predict L(T) values for training data
ytrainPred = []
for x in range(xtrain.shape[0]):
    ytrainPred.append(sum([xtrain.iat[x, z] * m[z] for z in range(xtrain.shape[1])]) + c)

#Predict L(T) values for testing data
ytestPred = []
for x in range(xtest.shape[0]):
    ytestPred.append(sum([xtest.iat[x, z] * m[z] for z in range(xtest.shape[1])]) + c)

In [11]:
#Denormalize training and testing data and predicted values
xtrain['L(T-3)'] = xtrain['L(T-3)'] * (x1max - x1min) + x1min
xtrain['L(T-2)'] = xtrain['L(T-2)'] * (x2max - x2min) + x2min
xtrain['L(T-1)'] = xtrain['L(T-1)'] * (x3max - x3min) + x3min
ytrain = ytrain * (ymax - ymin) + ymin
xtest['L(T-3)'] = xtest['L(T-3)'] * (x1max - x1min) + x1min
xtest['L(T-2)'] = xtest['L(T-2)'] * (x2max - x2min) + x2min
xtest['L(T-1)'] = xtest['L(T-1)'] * (x3max - x3min) + x3min
ytest = ytest * (ymax - ymin) + ymin
ytrainPred = np.multiply(ytrainPred, (ymax - ymin)) + ymin
ytestPred = np.multiply(ytestPred, (ymax - ymin)) + ymin

In [12]:
#Compute the error for training and testing data
MAE_Train = mean_absolute_error(ytrainPred, ytrain) #Testing MAE
MSE_Train = mean_squared_error(ytrainPred, ytrain) #Testing MSE
RMSE_Train = mean_squared_error(ytrainPred, ytrain) ** 0.5 #Testing RMSE
MAE_Test = mean_absolute_error(ytestPred, ytest) #Testing MAE
MSE_Test = mean_squared_error(ytestPred, ytest) #Testing MSE
RMSE_Test = mean_squared_error(ytestPred, ytest) ** 0.5 #Testing RMSE
print('Training Error :')
print('Mean Absolute Error :', MAE_Train)
print('Mean Squared Error :', MSE_Train)
print('Root Mean Squared Error :', RMSE_Train)
print('\nTesting Error :')
print('Mean Absolute Error :', MAE_Test)
print('Mean Squared Error :', MSE_Test)
print('Root Mean Squared Error :', RMSE_Test)

Training Error :
Mean Absolute Error : 442.35802979204766
Mean Squared Error : 425015.6395679606
Root Mean Squared Error : 651.9322354109825

Testing Error :
Mean Absolute Error : 435.4351422409806
Mean Squared Error : 382234.2423567807
Root Mean Squared Error : 618.2509541899476


In [13]:
#Analyze training data predictions
xtrain['Y_Train'] = ytrain
xtrain['Y_Train_Prediction'] = ytrainPred
xtrain

,L(T-3),L(T-2),L(T-1),Y_Train,Y_Train_Prediction
9,6292.87560,5349.80160,5225.40936,5000.47452,5138.779988
1535,6488.50500,6488.50500,6834.55860,6609.46788,6545.840384
895,6767.68608,5471.38800,5709.88440,5536.85760,5467.931209
1803,5024.01240,4972.57200,5208.10668,5744.48976,5419.458026
1056,5959.60416,6021.17676,5954.46012,5148.24876,5872.646650
...,...,...,...,...,...
599,5633.19144,5671.84968,5830.37964,6411.65616,5838.179715
1599,5900.99328,5442.86196,4595.34240,4412.18340,4656.587228
1361,5307.40224,4870.93824,4486.84992,4207.51296,4699.792421
1547,8223.60528,8138.65068,8665.36920,7292.37816,7794.156102


In [14]:
#Analyze testing data predictions
xtest['Y_Test'] = ytest
xtest['Y_Test_Prediction'] = ytestPred
xtest

,L(T-3),L(T-2),L(T-1),Y_Test,Y_Test_Prediction
1208,5352.13980,5505.05808,6541.50420,7238.13192,6550.183398
2117,4723.63164,5150.27520,6938.37468,7007.58540,7058.657859
1947,4451.93280,5377.23648,5097.89952,5848.61760,5448.567116
1128,5084.18208,5067.34704,5134.68720,5016.84192,5338.141983
2178,5045.52384,4626.51840,4567.90752,4723.63164,4835.854930
...,...,...,...,...,...
734,7647.47280,8196.17040,7943.64480,7908.72768,7271.626668
374,7098.77520,8607.69360,8607.69360,8590.54680,7998.307979
1931,6626.92644,4740.93432,5190.80400,4896.65844,5034.813246
140,6660.44064,6082.74936,6292.56384,6847.80840,6017.079747


In [15]:
#Deployment
x1 = float(input('Enter load at T-3 hours : '))
x2 = float(input('Enter load at T-2 hours : '))
x3 = float(input('Enter load at T-1 hours : '))
x1Norm = (x1 - x1min) / (x1max - x1min)
x2Norm = (x2 - x2min) / (x2max - x2min)
x3Norm = (x3 - x3min) / (x3max - x3min)
yPred = x1Norm * m[0] + x2Norm * m[1] + x3Norm * m[2] + c
y = yPred * (ymax - ymin) + ymin
print('Load at T hour :', y)

Enter load at T-3 hours : 6500
Enter load at T-2 hours : 6000
Enter load at T-1 hours : 7500
Load at T hour : 7151.193725300075
